In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load Dataset
df = pd.read_csv("/content/drive/MyDrive/Iris.csv")

# Convert Categorical into Numerical
df["Species"].replace(["Iris-setosa","Iris-versicolor","Iris-virginica"],[0,1,2],inplace=True)

# Assign Features and Label
Y = df["Species"].to_numpy()
X = df.drop(["Id","Species"],axis=1).to_numpy()

# Split Dataset into Train and Test Sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=720)

# Create External Functions
def affine_forward(X, weight, bias):
    out = np.dot(X,weight.T) + bias.T

    ldx = weight
    ldw = X
    ldb = 1
    return out,ldx,ldw,ldb

def affine_backward(ud, ldx, ldw, ldb):
    dx = np.dot(ud,ldx)
    dw = np.dot(ud.T,ldw)
    db = np.sum(ud,axis=0).reshape(-1, 1)
    return dx,dw,db

def gradient_decent(weight,der,alpha):

  return weight - alpha * der

def SVMloss_Func(logits,Y):

  diff = logits - logits[np.arange(Y.shape[0]),Y].reshape(Y.shape[0], 1)
  diff[diff < 0] = 0
  loss = np.sum(diff)
  diff[diff > 0 ] = 1
  diff[np.arange(Y.shape[0]), Y] = -1 * np.sum(diff, axis=1)
  return loss, diff

def relu_forward(X):
    X[X<0] = 0
    dx = np.zeros(X.shape)
    dx[X>0] = 1
    return X,dx

def relu_backward(ud, ld):
    return ud * ld

# Create Neural Network Class
class SingleLayerNeuralNetwork:
    def _init_(self, num_layers, neurons_per_layer):
        self.num_layers = num_layers
        self.neurons_per_layer = neurons_per_layer
        self.weights = None
        self.bias = None

    # Create a Train Function
    def train(self, X, Y, alpha, max_no_iter, loss_iter):
        # Initialize random weights and biases
        self.weights = []
        self.bias = []
        for i in range(self.num_layers):
            if i < self.num_layers - 1:
                weight = np.random.randn(self.neurons_per_layer[i], X.shape[1])
                bias = np.zeros((self.neurons_per_layer[i], 1))
            else:
                # For the output layer
                weight = np.random.randn(self.neurons_per_layer[i], self.neurons_per_layer[i-1])
                bias = np.zeros((self.neurons_per_layer[i], 1))
            self.weights.append(weight)
            self.bias.append(bias)
        for j in range(max_no_iter):
            logits = [X]
            ldx = []
            ldw = []
            ldb = []
            relu_dx = []
            # Forward pass (calculate logits)
            for i in range(self.num_layers):
                forward_out, ld_x, ld_w, ld_b = affine_forward(logits[-1],self.weights[i],self.bias[i])
                ldx.append(ld_x)
                ldw.append(ld_w)
                ldb.append(ld_b)
                relu_out, r_dx = relu_forward(forward_out)
                relu_dx.append(r_dx)
                logits.append(relu_out)

            # Calculate Loss
            loss , der  =  SVMloss_Func( logits[-1], Y)
            if (j + 1) % loss_iter == 0:
                # Print Loss Value
                print("After ", j+1 ,"Iteration, Loss: ",loss)

            # Backward pass
            backward_derivatives = [der]
            for i in range(self.num_layers - 1, -1, -1):
                #print(backward_derivatives[-1].shape)
                #print("Der",relu_dx[i].shape)
                der = relu_backward(backward_derivatives[-1], relu_dx[i])
                dx, dw, db = affine_backward(der, ldx[i], ldw[i], ldb[i])
                backward_derivatives.append(dx)
                # Update weights and biases
                self.weights[i] = gradient_decent(self.weights[i],dw,alpha)
                self.bias[i] = gradient_decent(self.bias[i],db,alpha)

    # Create a Test Function
    def test(self, X):
        logits = [X]
        ldx = []
        ldw = []
        ldb = []
        relu_dx = []
        for i in range(self.num_layers):
            forward_out, ld_x, ld_w, ld_b = affine_forward(logits[-1], self.weights[i], self.bias[i])
            ldx.append(ld_x)
            ldw.append(ld_w)
            ldb.append(ld_b)
            relu_out, r_dx = relu_forward(forward_out)
            relu_dx.append(r_dx)
            logits.append(relu_out)

        predicted_classes = np.argmax(relu_out, axis=1)
        return predicted_classes

# Create Object of the Neural Network class
SNn = SingleLayerNeuralNetwork(3, [4, 5, 3])

# Train the class
SNn.train(X_train, y_train, 0.00001, 1000, 1)

# Test the trained model
predictions = SNn.test(X_test)
accuracy = accuracy_score(y_test, predictions)
print("Test Accuracy:", accuracy)

TypeError: SingleLayerNeuralNetwork() takes no arguments